---
title: Discrimination Loss Fn
jupyter: python3
---

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import joblib
import re
import sys
sys.path.insert(0, '../')
import numpy as np

# Datasets
from aif360.datasets import MEPSDataset19
# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler

In [ ]:
dataset_orig_panel19_train = MEPSDataset19()

In [ ]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]]

In [ ]:
dataset_orig_panel19_train.feature_names[1]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Prepare data
X = dataset_orig_panel19_train.features
y = dataset_orig_panel19_train.labels.ravel()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = X_train
X_test_scaled = X_test

# Train logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Predict probabilities on the same scaled training data
train_probabilities = model.predict_proba(X_train_scaled)[:, 1]

# Calculation of discrimination index without modifying dataset structure
sens_attr_index = dataset_orig_panel19_train.feature_names.index('RACE')



# Define unprivileged and privileged values
unprivileged_val = 0.0
privileged_val = 1.0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()

# Custom loss function
def discrimination_loss(output, target, sensitive_features, lambda_val=10, k=2):
    criterion = nn.BCELoss()
    standard_loss = criterion(output, target)
    mask_unpriv = (sensitive_features == 0)
    mask_priv = (sensitive_features == 1)
    prob_unpriv = torch.mean(output[mask_unpriv])
    prob_priv = torch.mean(output[mask_priv])

    discrimination = lambda_val * (prob_priv - prob_unpriv) ** k

    loss_val=(1 + lambda_val * discrimination) * standard_loss
    return loss_val,discrimination.item() 

def calculate_accuracy(predictions, targets):
    predicted_classes = (predictions >= 0.5).float()
    return (predicted_classes == targets).float().mean()


data = torch.tensor(X_train_scaled).float()
targets = torch.tensor(y_train).float().unsqueeze(1)

# Correctly preparing the sensitive features
threshold = 0.5  # Adjust the threshold according to your specific case
sensitive_features = torch.tensor((data[:, 1].numpy() > threshold).astype(float)).float()
features = torch.cat((data[:, :1], data[:, 2:]), dim=1)

# Assuming similar preparation for test data
test_data = torch.tensor(X_test_scaled).float()
test_targets = torch.tensor(y_test).float().unsqueeze(1)
test_sensitive_features = torch.tensor((test_data[:, 1].numpy() > threshold).astype(float)).float()
test_features = torch.cat((test_data[:, :1], test_data[:, 2:]), dim=1)

model2 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model2.parameters(), lr=0.01)
train_losses, train_accuracies, train_discriminations,train_fairness = [], [], [],[]
test_losses, test_accuracies, test_discriminations,test_fairness = [], [], [],[]

# Training loop
model2.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model2(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model2.eval()
    with torch.no_grad():
        test_outputs = model2(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses.append(loss.item())
    train_accuracies.append(train_accuracy.item() * 100)
    train_discriminations.append(discrimination)
    fairness=1-discrimination
    train_fairness.append(fairness)
    test_fairness.append(fairness)
    test_losses.append(test_loss.item())
    test_accuracies.append(test_accuracy.item() * 100)
    test_discriminations.append(test_discrimination)
    model2.train()

#### MODEL ACCROSS DIFFERENT VALUES OF LAMBDA

#### LAMBDA < 1


In [ ]:
model3 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model3.parameters(), lr=0.01)
train_losses3, train_accuracies3, train_discriminations3,train_fairness3 = [], [], [],[]
test_losses3, test_accuracies3, test_discriminations3,test_fairness3 = [], [], [],[]

# Training loop
model3.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model3(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=0.01, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model3.eval()
    with torch.no_grad():
        test_outputs = model3(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=0.01, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses3.append(loss.item())
    train_accuracies3.append(train_accuracy.item() * 100)
    train_discriminations3.append(discrimination)
    fairness=1-discrimination
    train_fairness3.append(fairness)
    test_fairness3.append(fairness)
    test_losses3.append(test_loss.item())
    test_accuracies3.append(test_accuracy.item() * 100)
    test_discriminations3.append(test_discrimination)
    model3.train()

#### LAMBDA = 1


In [ ]:
model4 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model4.parameters(), lr=0.01)
train_losses4, train_accuracies4, train_discriminations4,train_fairness4 = [], [], [],[]
test_losses4, test_accuracies4, test_discriminations4,test_fairness4 = [], [], [],[]

# Training loop
model4.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model4(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=1, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model4.eval()
    with torch.no_grad():
        test_outputs = model4(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=1, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses4.append(loss.item())
    train_accuracies4.append(train_accuracy.item() * 100)
    train_discriminations4.append(discrimination)
    fairness=1-discrimination
    train_fairness4.append(fairness)
    test_fairness4.append(fairness)
    test_losses4.append(test_loss.item())
    test_accuracies4.append(test_accuracy.item() * 100)
    test_discriminations4.append(test_discrimination)
    model4.train()

#### LAMBDA > 1


In [ ]:
model5 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model5.parameters(), lr=0.01)
train_losses5, train_accuracies5, train_discriminations5,train_fairness5 = [], [], [],[]
test_losses5, test_accuracies5, test_discriminations5,test_fairness5 = [], [], [],[]

# Training loop
model5.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model5(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=10, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model5.eval()
    with torch.no_grad():
        test_outputs = model5(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=10, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses5.append(loss.item())
    train_accuracies5.append(train_accuracy.item() * 100)
    train_discriminations5.append(discrimination)
    fairness=1-discrimination
    train_fairness5.append(fairness)
    test_fairness5.append(fairness)
    test_losses5.append(test_loss.item())
    test_accuracies5.append(test_accuracy.item() * 100)
    test_discriminations5.append(test_discrimination)
    model5.train()

#### LAMBDA >> 1


In [ ]:
model6 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model6.parameters(), lr=0.01)
train_losses6, train_accuracies6, train_discriminations6,train_fairness6 = [], [], [],[]
test_losses6, test_accuracies6, test_discriminations6,test_fairness6 = [], [], [],[]

# Training loop
model6.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model6(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=100, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model6.eval()
    with torch.no_grad():
        test_outputs = model6(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=100, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses6.append(loss.item())
    train_accuracies6.append(train_accuracy.item() * 100)
    train_discriminations6.append(discrimination)
    fairness=1-discrimination
    train_fairness6.append(fairness)
    test_fairness6.append(fairness)
    test_losses6.append(test_loss.item())
    test_accuracies6.append(test_accuracy.item() * 100)
    test_discriminations6.append(test_discrimination)
    model6.train()

#### MODEL ACCROSS DIFFERENT VALUES OF K

### K=2


In [ ]:
model7 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model7.parameters(), lr=0.01)
train_losses7, train_accuracies7, train_discriminations7,train_fairness7 = [], [], [],[]
test_losses7, test_accuracies7, test_discriminations7,test_fairness7 = [], [], [],[]

# Training loop
model7.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model7(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=100, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model7.eval()
    with torch.no_grad():
        test_outputs = model7(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=100, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses7.append(loss.item())
    train_accuracies7.append(train_accuracy.item() * 100)
    train_discriminations7.append(discrimination)
    fairness=1-discrimination
    train_fairness7.append(fairness)
    test_fairness7.append(fairness)
    test_losses7.append(test_loss.item())
    test_accuracies7.append(test_accuracy.item() * 100)
    test_discriminations7.append(test_discrimination)
    model7.train()


### K=3


In [ ]:
model8 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model8.parameters(), lr=0.01)
train_losses8, train_accuracies8, train_discriminations8,train_fairness8 = [], [], [],[]
test_losses8, test_accuracies8, test_discriminations8,test_fairness8 = [], [], [],[]

# Training loop
model8.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model8(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=100, k=3)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model8.eval()
    with torch.no_grad():
        test_outputs = model8(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=100, k=3)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses8.append(loss.item())
    train_accuracies8.append(train_accuracy.item() * 100)
    train_discriminations8.append(discrimination)
    fairness=1-discrimination
    train_fairness8.append(fairness)
    test_fairness8.append(fairness)
    test_losses8.append(test_loss.item())
    test_accuracies8.append(test_accuracy.item() * 100)
    test_discriminations8.append(test_discrimination)
    model8.train()


#### MODEL ACCROSS DIFFERENT VALUES OF LAMBDA


In [ ]:
import plotly.graph_objects as go

def plot_comp_metric(title, y_label, val1, val2, val3, val4):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    if val1:
        fig.add_trace(go.Scatter(
            x=epochs, y=val1, mode='lines+markers',
            name='Lambda = 0.1',
            line=dict(color='RoyalBlue', width=2),
            marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))
    if val2:

        # Adding Test Line with Markers
        fig.add_trace(go.Scatter(
            x=epochs, y=val2, mode='lines+markers',
            name='lambda = 1',
            line=dict(color='Crimson', width=2, dash='dot'),
            marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))
    # Adding Test Line with Markers
    if val3:
        fig.add_trace(go.Scatter(
            x=epochs, y=val3, mode='lines+markers',
            name='lambda = 10',
            line=dict(color='Pink', width=2, dash='dot'),
            marker=dict(color='Pink', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))

    if val4:
     # Adding Test Line with Markers
        fig.add_trace(go.Scatter(
            x=epochs, y=val4, mode='lines+markers',
            name='lambda = 100',
            line=dict(color='Purple', width=2, dash='dot'),
            marker=dict(color='Purple', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))


    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Epoch',
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

In [ ]:
plot_comp_metric("TEST_Fairness", "Fairness", test_fairness3, test_fairness4, test_fairness5, test_fairness6)
plot_comp_metric("TEST_Discriminations", "Discrimination", test_discriminations3, test_discriminations4, test_discriminations5, test_discriminations6)
plot_comp_metric("TEST_Losses", "Losses", test_losses3,test_losses4,test_losses5,test_losses6)

#### MODEL ACCROSS DIFFERENT VALUES OF K


In [ ]:
import plotly.graph_objects as go

def plot_comp_metric_k(title, y_label, val1, val2):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    if val1:
        fig.add_trace(go.Scatter(
            x=epochs, y=val1, mode='lines+markers',
            name='K = 2',
            line=dict(color='RoyalBlue', width=2),
            marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))
    if val2:

        # Adding Test Line with Markers
        fig.add_trace(go.Scatter(
            x=epochs, y=val2, mode='lines+markers',
            name='K = 3',
            line=dict(color='Crimson', width=2, dash='dot'),
            marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))



    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Epoch',
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

In [ ]:
plot_comp_metric_k("TEST_Fairness", "Fairness", test_fairness7, test_fairness8)
plot_comp_metric_k("TEST_Discriminations", "Discrimination", test_discriminations7, test_discriminations8)
plot_comp_metric_k("TEST_Losses", "Losses", test_losses7,test_losses8)

#### LOSS ACCURACY AND DISCRIMINATION FOR MODEL WITH DISCRIMINATION FUNCTION


In [ ]:
import plotly.graph_objects as go

def plot_metric(title, y_label, train_data, test_data):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=train_data, mode='lines+markers',
        name='Train',
        line=dict(color='RoyalBlue', width=2),
        marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Adding Test Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=test_data, mode='lines+markers',
        name='Test',
        line=dict(color='Crimson', width=2, dash='dot'),
        marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Epoch',
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

In [ ]:
# Example usage
plot_metric("Training and Testing Loss", "Loss", train_losses, test_losses)
plot_metric("Training and Testing Accuracy", "Accuracy (%)", train_accuracies, test_accuracies)
plot_metric("Training and Testing Discrimination", "Discrimination", train_discriminations, test_discriminations)
plot_metric("Training and Testing Fairness", "Fairness", train_fairness, test_fairness)

## BCE Loss Fn


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()

# Custom loss function|
def discrimination_loss(output, target, sensitive_features, lambda_val=100, k=2):
    criterion = nn.BCELoss()
    standard_loss = criterion(output, target)

    mask_unpriv = (sensitive_features == 0)
    mask_priv = (sensitive_features == 1)
    #discrimination=torch.abs(prob_priv)
    prob_unpriv = torch.mean(output[mask_unpriv])
    prob_priv = torch.mean(output[mask_priv])
    discrimination = lambda_val*(prob_priv - prob_unpriv) ** k

# Handle cases where one group might be missing
    #discrimination=torch.abs(prob_priv)
 
    loss_val=standard_loss

    return loss_val,discrimination.item() 

def calculate_accuracy(predictions, targets):
    predicted_classes = (predictions >= 0.5).float()
    return (predicted_classes == targets).float().mean()


data = torch.tensor(X_train_scaled).float()
targets = torch.tensor(y_train).float().unsqueeze(1)

# Correctly preparing the sensitive features
threshold = 0.5  # Adjust the threshold according to your specific case
sensitive_features = torch.tensor((data[:, 1].numpy() > threshold).astype(float)).float()
features = torch.cat((data[:, :1], data[:, 2:]), dim=1)

# Assuming similar preparation for test data
test_data = torch.tensor(X_test_scaled).float()
test_targets = torch.tensor(y_test).float().unsqueeze(1)
test_sensitive_features = torch.tensor((test_data[:, 1].numpy() > threshold).astype(float)).float()
test_features = torch.cat((test_data[:, :1], test_data[:, 2:]), dim=1)

model = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.01)
train_losses1, train_accuracies1, train_discriminations1,train_fairness1 = [], [], [],[]
test_losses1, test_accuracies1, test_discriminations1,test_fairness1 = [], [], [],[]
# Training loop
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses1.append(loss.item())
    train_accuracies1.append(train_accuracy.item() * 100)
    train_discriminations1.append(discrimination)
    fairness=1-discrimination
    train_fairness1.append(fairness)
    test_fairness1.append(fairness)
    test_losses1.append(test_loss.item())
    test_accuracies1.append(test_accuracy.item() * 100)
    test_discriminations1.append(test_discrimination)
    model.train()

In [ ]:
import plotly.graph_objects as go


def plot_metric(title, y_label, train_data, test_data, x_title = "Epoch", epoch_blue = "train", epoch_red = "Test"):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=train_data, mode='lines+markers',
        name=epoch_blue,
        line=dict(color='RoyalBlue', width=2),
        marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Adding Test Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=test_data, mode='lines+markers',
        name=epoch_red,
        line=dict(color='Crimson', width=2, dash='dot'),
        marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title=x_title,
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

# Example usage
plot_metric("Training and Testing Loss", "Loss", train_losses1, test_losses1)
plot_metric("Training and Testing Accuracy", "Accuracy (%)", train_accuracies1, test_accuracies1)
plot_metric("Training and Testing Discrimination", "Discrimination", train_discriminations1, test_discriminations1)
plot_metric("Training and Testing Fairness", "Fairness", train_fairness1, test_fairness1)

In [ ]:
plot_metric("DISPARITY IN FAIRNESS", "FAIRNESS", test_fairness, test_fairness1, "EPOCH", "WITH DISCRIMINATION FUNCTION", "BCE")